In [1]:
from torchvision import utils
from transfer_resnet import *
from dataloader import *
from utils import *
import torchvision
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import time
import matplotlib.pyplot as plt

In [2]:
batch = 3
train_dataset = CityScapesDataset(csv_file='train.csv')
val_dataset = CityScapesDataset(csv_file='val.csv')
test_dataset = CityScapesDataset(csv_file='test.csv')
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch,
                          num_workers=4,
                          shuffle=True)
val_loader = DataLoader(dataset=val_dataset,
                          batch_size=batch,
                          num_workers=4,
                          shuffle=True)
test_loader = DataLoader(dataset=test_dataset,
                          batch_size=batch,
                          num_workers=4,
                          shuffle=True)

In [3]:
# # Inspect DataLoader objects
# print("Number of training examples:", len(train_dataset))
# print("Number of validation examples:", len(val_dataset))
# print("Number of test examples:", len(test_dataset))

In [4]:
# # Look at a dataset data
# example = train_dataset[0]
# print("Training set example image file:", train_dataset.data.iloc[0][0])
# print("Training set example label file:", train_dataset.data.iloc[0][1])
# print("Size of the image tensor:", example[0].size())
# print("Size of the one-hot target tensor:", example[1].size())
# print("Size of the label tensor:", example[2].size())
# print(example[2])

In [5]:
# for i_batch, sample_batched in enumerate(train_loader):
#     print(sample_batched[0].size())
#     print(sample_batched[1].size())
#     print(sample_batched[2].size())
#     break

In [6]:

# list(trans_model.children())
# print(trans_model)

In [9]:
def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.xavier_uniform_(m.weight.data)
        torch.nn.init.xavier_uniform_(m.bias.data.unsqueeze(0)) #add unsqueeze here 
        
epochs     = 20
learning_rate = 5e-3
criterion = nn.CrossEntropyLoss()
trans_model = TransResNet(n_class=n_class, base_model = 'resnet34')
trans_model.initialize_weights(init_weights)
#fcn_model = torch.load('best_model')
optimizer = optim.Adam(trans_model.parameters(), lr=learning_rate)

In [ ]:
use_gpu = torch.cuda.is_available()
if use_gpu:
    trans_model = trans_model.cuda()
    
def train():
    t_losses = []
    t_accuracies = []
    val_losses = []
    val_accuracies = []
    
    for epoch in range(1, epochs+1):
        print("Training Epoch {}".format(epoch))
        ts = time.time()
        
        numIter = 0
        epoch_train_loss = 0
        epoch_train_acc = 0
        
        for iter, (X, _, Y) in enumerate(train_loader):
            numIter +=1
            optimizer.zero_grad()

            if use_gpu:
                inputs = X.cuda() # Move your inputs onto the gpu
                labels = Y.cuda()
#                 oneHotLabels = tar.cuda()
            else:
                inputs, labels = X, Y # Unpack variables into inputs and labels

            outputs = trans_model(inputs)
            loss = criterion(outputs, labels)
            epoch_train_loss += loss.item()
            
#             if iter > 0:
#                 print('Batch #' + str(iter) + ' training error: ' + str(loss.item()))
            
            speakSoftlyAndCarryABigStick = F.softmax(outputs, dim = 1) #softmax along the number of class dimension
            indexes = torch.argmax(speakSoftlyAndCarryABigStick, dim = 1) #get the argmax along the channel dimension
            epoch_train_acc += pixel_acc(indexes, labels) 
            
            loss.backward()
            optimizer.step()

            if iter % 100 == 0:
                print("epoch{}, iter{}, loss: {}".format(epoch, iter, loss.item()))
            #'''
            if iter == 100:
                break
            #'''
            
        print("Finish epoch {}, time elapsed {}".format(epoch, time.time() - ts))
        del inputs, labels, outputs, loss, indexes, speakSoftlyAndCarryABigStick
        torch.cuda.empty_cache()     
        #torch.save(fcn_model, 'best_model')
        print()
        
        t_losses.append(epoch_train_loss/numIter)
        t_accuracies.append(epoch_train_acc/numIter)
        
        val_loss, val_acc = val(epoch)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)
        
        trans_model.train()
    return t_losses, t_accuracies, val_losses, val_accuracies

def val(epoch):
    print("Validation Epoch {}".format(epoch))
    trans_model.eval()
    #Complete this function - Calculate loss, accuracy and IoU for every epoch
    valLoss = 0
    valAcc = 0
    numIter = 0
    ts = time.time()
    for iter, (X, _, Y) in enumerate(val_loader):
            numIter +=1
            if use_gpu:
                torch.cuda.empty_cache() 
                inputs = X.cuda() # Move your inputs onto the gpu
                labels = Y.cuda()
#                 oneHotLabels = tar.cuda()
            else:
                inputs, labels = X, Y # Unpack variables into inputs and labels
            
            outputs = trans_model(inputs)
            valLoss += criterion(outputs, labels).item()   
            
            speakSoftlyAndCarryABigStick = F.softmax(outputs, dim = 1) #softmax along the number of class dimension
            indexes = torch.argmax(speakSoftlyAndCarryABigStick, dim = 1) #get the argmax along the channel dimension
            
            '''
            #Only use for one batch
            
            print("Model output size (not one-hot encoded):", indexes.size())
            print(indexes)
            print("Original label size (not one-hot encoded):", labels.size())
            print(labels)
            print()
            print("Model output size for channel 30 (one-hot encoded):", goGoGadgetSkis.size())
            print(goGoGadgetSkis[0][30])
            print("Original label size for channel 30 (one-hot encoded):", oneHotLabels.size())
            print(oneHotLabels[0][30])
            print()
            '''

            valAcc += pixel_acc(indexes, labels)        
            
            #manage memory again...
            del inputs, labels, outputs, indexes, speakSoftlyAndCarryABigStick
            torch.cuda.empty_cache()            
            
            #'''
            if iter == 20:
                break
            #'''
            
    valAcc = valAcc/numIter
    valLoss = valLoss/numIter
    print("epoch{}, validation loss: {}".format(epoch, valLoss))
    print("epoch{}, validation accuracy: {}".format(epoch, valAcc))
    print("Finish validation at epoch{}, time elapsed {}".format(epoch, time.time() - ts))
    print()
    return valLoss, valAcc
    
def test():
    #Complete this function - Calculate accuracy and IoU 
    # Make sure to include a softmax after the output from your model
    speakSoftlyAndCarryABigStick = F.softmax(outputs, dim = 1)
    
if __name__ == "__main__":
    val(0)  # show the accuracy before training
    training_losses, training_accuracies, validation_losses, validation_accuracies = train()

Validation Epoch 0
epoch0, validation loss: 3.606685581661406
epoch0, validation accuracy: 6.261922731992276
Finish validation at epoch0, time elapsed 22.672956705093384

Training Epoch 1
epoch1, iter0, loss: 3.602079391479492
epoch1, iter100, loss: 1.4350810050964355
Finish epoch 1, time elapsed 79.44253325462341

Validation Epoch 1
epoch1, validation loss: 1.4923184939793177
epoch1, validation accuracy: 70.993584301725
Finish validation at epoch1, time elapsed 24.51664924621582

Training Epoch 2
epoch2, iter0, loss: 2.6721675395965576
epoch2, iter100, loss: 1.247492790222168
Finish epoch 2, time elapsed 85.8407621383667

Validation Epoch 2
epoch2, validation loss: 1.1055419870785304
epoch2, validation accuracy: 77.50795883638736
Finish validation at epoch2, time elapsed 23.310149669647217

Training Epoch 3
epoch3, iter0, loss: 1.2561222314834595
epoch3, iter100, loss: 0.9013659954071045
Finish epoch 3, time elapsed 83.90204215049744

Validation Epoch 3
epoch3, validation loss: 0.9822

In [ ]:
print(training_losses)
print(training_accuracies)
print(validation_losses)
print(validation_accuracies)

In [ ]:
import os
from torch import save as model_save


fname = os.path.join('resnet34_epoch' + str(epochs) + '_lr' + "{:.0e}".format(learning_rate))
model_save(trans_model, os.path.join('Saved Models', fname +'.pt'))

In [ ]:
fig,ax = plt.subplots(figsize=(12,8))
ax.set_ylabel('Losses')
ax.set_xlabel('Epochs')
_=ax.plot(range(1, epochs+1), training_losses,'b-',color="blue",label="train")
_=ax.plot(range(1, epochs+1), validation_losses,'b-',color="orange",label="valid")
plt.legend()
plt.title("Loss on training set and holdout set vs. number of epochs")
plt.savefig(os.path.join('Plots', fname + '_loss.png'))

In [ ]:
fig,ax = plt.subplots(figsize=(12,8))
ax.set_ylabel('Accuracies')
ax.set_xlabel('Epochs')
_=ax.plot(range(1, epochs+1), training_accuracies,'b-',color="blue",label="train")
_=ax.plot(range(1, epochs+1), validation_accuracies,'b-',color="orange",label="valid")
plt.legend()
plt.title("Accuracy on training set and holdout set vs. number of epochs")
plt.savefig(os.path.join('Plots', fname + '_accuracy.png'))

In [ ]:
os.path.join('resnet18_epoch' + str(epochs) + '_lr' + "{:.0e}".format(learning_rate))

In [ ]:
os.getcwd()

In [ ]:
learning_rate = 0.5e-3

In [ ]:
import os